### 交差点の緯度経度をポチポチするUI

In [ ]:
!pip install streamlit streamlit-folium folium pandas

In [ ]:
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import st_folium
import ast
import os

In [ ]:
# --- 設定 ---
CSV_FILE = "final_landmark_results.csv"  # 読み込むファイル名
SAVE_FILE = "corrected_landmarks.csv"    # 保存するファイル名（上書き防止で別名推奨）

# ページ設定
st.set_page_config(layout="wide", page_title="交差点修正ツール")

# --- 関数: データの読み込み ---
@st.cache_data
def load_data():
    if os.path.exists(SAVE_FILE):
        df = pd.read_csv(SAVE_FILE)
    elif os.path.exists(CSV_FILE):
        df = pd.read_csv(CSV_FILE)
    else:
        st.error(f"ファイルが見つかりません: {CSV_FILE}")
        return None

    # 文字列として保存されているリストを、Pythonのリストオブジェクトに戻す
    # (CSVに保存するとリストは "[...]" というただの文字になるため)
    if 'landmarks_with_intersections' in df.columns:
        # 安全に評価する
        df['landmarks_with_intersections'] = df['landmarks_with_intersections'].apply(
            lambda x: ast.literal_eval(x) if isinstance(x, str) else x
        )
    return df

# --- メイン処理 ---
def main():
    st.title("📍 交差点位置 手動修正ツール")
    st.markdown("自動検出された交差点の位置がずれている場合、地図をクリックして修正できます。")

    # データのロード
    if 'df' not in st.session_state:
        st.session_state.df = load_data()

    df = st.session_state.df
    if df is None:
        return

    # --- 1. 店舗の選択 ---
    # サイドバーで操作
    st.sidebar.header("検索・選択")
    
    # ページネーションのようなUI
    row_index = st.sidebar.number_input("行番号 (Index) を選択", min_value=0, max_value=len(df)-1, value=0, step=1)
    
    # 選択された行のデータ
    row = df.iloc[row_index]
    landmarks = row['landmarks_with_intersections']

    st.sidebar.markdown(f"**店舗座標**: {row['lat']:.4f}, {row['lng']:.4f}")
    
    # ランドマークがない場合
    if not isinstance(landmarks, list) or len(landmarks) == 0:
        st.warning("この行にはランドマーク情報がありません。")
        return

    # --- 2. ランドマークの選択 ---
    landmark_names = [lm.get('name', '不明') for lm in landmarks]
    selected_lm_index = st.sidebar.radio("修正するランドマークを選択", range(len(landmark_names)), format_func=lambda x: landmark_names[x])
    
    target_lm = landmarks[selected_lm_index]
    
    # 現在の交差点情報
    current_intersection = target_lm.get('nearest_intersection')
    
    # --- 3. 地図の表示 ---
    col1, col2 = st.columns([2, 1])
    
    with col1:
        st.subheader(f"🗺️ 地図: {target_lm.get('name')}")
        
        # 地図の中心：現在の交差点があればそこ、なければランドマーク自体、それもなければ店舗
        if current_intersection:
            center_lat = current_intersection['intersection_lat']
            center_lon = current_intersection['intersection_lon']
        else:
            center_lat = target_lm['lat']
            center_lon = target_lm['lon']

        m = folium.Map(location=[center_lat, center_lon], zoom_start=17)

        # A. 店舗（青）
        folium.Marker(
            [row['lat'], row['lng']], 
            popup="店舗位置", 
            icon=folium.Icon(color="blue", icon="home")
        ).add_to(m)

        # B. ランドマーク自体の位置（緑）
        folium.Marker(
            [target_lm['lat'], target_lm['lon']], 
            tooltip=f"ランドマーク: {target_lm['name']}", 
            icon=folium.Icon(color="green", icon="flag")
        ).add_to(m)

        # C. 自動検出された交差点（赤）
        if current_intersection:
            folium.Marker(
                [current_intersection['intersection_lat'], current_intersection['intersection_lon']], 
                popup="現在の交差点（自動検出）",
                icon=folium.Icon(color="red", icon="exclamation-sign")
            ).add_to(m)

        # ★重要: クリックイベントを取得する
        map_data = st_folium(m, height=500, width="100%")

    # --- 4. 修正処理 ---
    with col2:
        st.subheader("🛠️ 修正操作")
        
        st.info("👈 地図上の「正しい交差点」をクリックしてください。")

        # クリックされた座標を取得
        if map_data['last_clicked']:
            clicked_lat = map_data['last_clicked']['lat']
            clicked_lng = map_data['last_clicked']['lng']
            
            st.write("📍 **クリックされた座標**")
            st.code(f"Lat: {clicked_lat:.6f}\nLon: {clicked_lng:.6f}")
            
            # 保存ボタン
            if st.button("この座標で更新する", type="primary"):
                # データを更新
                new_intersection_data = {
                    "intersection_lat": clicked_lat,
                    "intersection_lon": clicked_lng,
                    "street_count": 99, # 手動修正フラグとして適当な数字を入れるか、別途フラグを立てる
                    "is_manual_fix": True # 手動修正フラグ
                }
                
                # メモリ上のデータを書き換え
                st.session_state.df.iloc[row_index]['landmarks_with_intersections'][selected_lm_index]['nearest_intersection'] = new_intersection_data
                
                # CSVに即時保存
                st.session_state.df.to_csv(SAVE_FILE, index=False)
                
                st.success("✅ 更新して保存しました！")
                st.rerun() # 画面リロードして反映
        else:
            st.write("（地図をクリックするとここに座標が表示されます）")

        st.markdown("---")
        st.markdown("**現在のデータ状況**")
        if current_intersection:
            if current_intersection.get('is_manual_fix'):
                st.write("🟢 **手動修正済み**")
            else:
                st.write("🤖 自動検出データのまま")
        else:
            st.write("❌ 交差点データなし")

if __name__ == "__main__":
    main()
